In [1]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd


def manual_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    # if you are suing GPU
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def get_model_size(model):
	total_size = sum(param.numel() for param in model.parameters() if param.requires_grad)
	return total_size / 1e6

class Config:
    epochs = 5
    lr = 1e-4
    warmup = 0
    
    n_nodes = 255
    n_feats = 6
    
    eval_steps = 256
    
    seq_len = 128
    horizon = 32
  
    # myrnn
    d_latent: int = 32             # latent channels d
    delta_t: float = 0.5                # time step
    device: str = "cuda"
    
    seed = 42

args = Config()
manual_seed(args.seed)


In [2]:
import pandas as pd
import numpy as np


def make_tensor(df, feature_cols=None):
    if feature_cols is None:
        feature_cols = df.columns.drop(['Date', 'Ticker']).tolist()

    # 1) ép kiểu & sort
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker','Date'])

    # (tuỳ chọn) xử lý missing: forward-fill theo từng ticker
    # df[feature_cols] = df.groupby('Ticker', group_keys=False)[feature_cols].ffill()

    # 2) xác định thứ tự ticker & ngày
    tickers = sorted(df['Ticker'].unique())
    dates   = sorted(df['Date'].unique())

    # 3) pivot từng feature rồi stack vào trục cuối
    mats = []
    for feat in feature_cols:
        wide = (
            df.pivot_table(index='Ticker', columns='Date', values=feat, aggfunc='first')
              .reindex(index=tickers, columns=dates)
        )
        mats.append(wide.values)  # shape: (n_tickers, n_days)

    # 4) stack thành tensor: (n_tickers, n_days, n_features)
    X = np.stack(mats, axis=-1).astype('float32')

    return X, tickers, dates, feature_cols
  

def get_data_splits(path='../input/sp500_2009_2025/sp500_prices_clean.csv'):
    df = pd.read_csv(path)
    
    raw_data, tickers, dates, feature_cols = make_tensor(df)

    data = np.log(1 + raw_data)
    training_data = data[:, :-30, :]
    testing_data = data[:, -30:, :]
    
    return training_data, testing_data

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset

class StockDataset(Dataset):
    def __init__(self, data, ws=128):
        self.data = data
        self.ws = ws
        self.samples = []
        
        self.n_tickers, self.n_days, self.n_features = self.data.shape
        
        for start in range(self.n_days - self.ws + 1):
            self.samples.append(start)
            
    def __len__(self):
        return len(self.samples)
      
    def __getitem__(self, idx):
        start = self.samples[idx]
        x = torch.tensor(self.data[:, start:start + self.ws], dtype=torch.float32)
        return x
    

In [4]:
training_data_np, testing_data_np = get_data_splits()
train_dataset = StockDataset(training_data_np, ws=128)
train_dataloader = DataLoader(train_dataset, batch_size=1)

# Model

In [5]:

from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyRNN(nn.Module):
    def __init__(self, n_nodes: int, n_feats: int, args: Config):
        super().__init__()
        self.n_nodes = n_nodes
        self.rank_A = 32
        self.n_feats = n_feats
        self.args = args
        self.d_latent = getattr(args, "d_latent", 32)
        self.dropout = 0.0
        self.n_layers = 1
        
        
        self.cells = nn.ModuleList([
            nn.LSTMCell(n_feats if i == 0 else self.d_latent, self.d_latent)
            for i in range(self.n_layers)
        ])
        
        self.readout = nn.Sequential(
            nn.Linear(self.d_latent, self.d_latent), nn.ReLU(),
            nn.Linear(self.d_latent, n_feats)
        )
        
        self.enc_in = None
        self.static_node_features = nn.Parameter(torch.randn(n_nodes, self.d_latent) * 0.1)

    def _init_states(
        self, X0: torch.Tensor, H0: Optional[torch.Tensor]
    ):
        """
        Trả về list (h_list, c_list) cho từng layer.
        - Nếu H0 cung cấp (N, d), đặt h_top = H0; ngược lại nếu enc_in != None thì h_top = enc(X0); còn lại h=0.
        - c luôn = 0.
        """
        device = X0.device
        dtype = X0.dtype
        N = X0.shape[0]

        h_list = []
        c_list = []
        for _ in range(self.n_layers):
            h_list.append(self.static_node_features.clone())
            c_list.append(self.static_node_features.clone())

        if H0 is not None:
            h_list[-1] = H0.to(device=device, dtype=dtype)
        elif self.enc_in is not None:
            h_list[-1] = self.enc_in(X0)

        return h_list, c_list

    def _step(self, x_t: torch.Tensor, h_list, c_list):
        """
        Chạy 1 bước LSTM qua tất cả các layer.
        Input layer nhận x_t; các layer sau nhận h của layer trước ở cùng time-step.
        Trả về (h_list_new, c_list_new, h_top).
        """
        inp = x_t
        new_h, new_c = [], []
        for l, cell in enumerate(self.cells):
            h_t, c_t = cell(inp, (h_list[l], c_list[l]))
            new_h.append(h_t)
            new_c.append(c_t)
            inp = h_t
        h_top = new_h[-1]
        if self.training and self.dropout > 0:
            h_top = F.dropout(h_top, p=self.dropout, training=True)
        return new_h, new_c, h_top


    def forward(
        self,
        X: torch.Tensor,
        H0: Optional[torch.Tensor] = None,
        horizon: int = 0,
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Deterministic inference.
        Args:
            X: (N, T, F)
            H0: (N, d_latent) optional initial hidden for top layer
            horizon: số bước rollout sau quan sát cuối
        Returns:
            y_all: (N, T-1 + horizon, F)  # dự báo X_{t+1}
            r_all: (N, T-1 + horizon, F)  # residual
            H_all: (N, T + horizon, d)    # hidden top-layer (bao gồm bước khởi tạo tại index 0)
        """
        device = next(self.parameters()).device
        X = X.to(device)
        N, T, Fdim = X.shape

        # Khởi tạo trạng thái
        X0 = X[:, 0, :]
        h_list, c_list = self._init_states(X0, H0)

        # Khởi tạo output tensors
        ext = max(horizon, 0)
        H_all = torch.zeros(N, T + ext, self.d_latent, device=device, dtype=X.dtype)
        y_all = torch.zeros(N, (T - 1) + ext, Fdim, device=device, dtype=X.dtype)
        r_all = torch.zeros_like(y_all)

        # Lưu hidden ban đầu (tương tự H_t trước khi update lần đầu)
        H_all[:, 0] = h_list[-1]

        # Quan sát: t = 0..T-2 (dự báo X_{t+1})
        for t in range(T - 1):
            x_t = X[:, t, :]  # (N, F)
            h_list, c_list, h_top = self._step(x_t, h_list, c_list)
            r_t = self.readout(h_top)
            y_t = x_t + r_t

            H_all[:, t + 1] = h_top
            r_all[:, t] = r_t
            y_all[:, t] = y_t

        # Rollout autoregressive nếu cần
        if horizon > 0:
            x_t = X[:, -1, :]
            for s in range(horizon):
                h_list, c_list, h_top = self._step(x_t, h_list, c_list)
                r_t = self.readout(h_top)
                y_t = x_t + r_t

                idx = (T - 1) + s
                y_all[:, idx] = y_t
                r_all[:, idx] = r_t
                H_all[:, T + s] = h_top

                x_t = y_t  # autoregressive

        return y_all, r_all, H_all

    def forward(self, X: torch.Tensor,
                H0: Optional[torch.Tensor] = None,
                horizon: int = 0) -> Tuple[torch.Tensor, torch.Tensor]:
        """Run deterministic inference.
        Args:
            X: (N, T, F) input features at each t
            H0: (N, d) optional initial latent state (default zeros)
            Atilde, L: if precomputed; otherwise build from X
        Returns:
            y_pred: (N, T-1) predicted next-day log-return for each node
            H_all: (N, T, d) latent states (including H0 as first)
        """
        device = self.args.device
        X = X.to(next(self.parameters()).device)
        n_nodes, n_steps, n_feats = X.shape
        d_latent = getattr(self.args, "d_latent", 64)
        

        X0 = X[:, 0, :] # (N, F)
        h_list, c_list = self._init_states(X0, H0)

        H_all = torch.zeros(n_nodes, n_steps + max(horizon, 0), d_latent, device=device, dtype=X.dtype)
        r_all = torch.zeros(n_nodes, n_steps - 1 + max(horizon, 0), n_feats, device=device, dtype=X.dtype)
        y_all = torch.zeros(n_nodes, n_steps - 1 + max(horizon, 0), n_feats, device=device, dtype=X.dtype)


        # Lưu hidden ban đầu (tương tự H_t trước khi update lần đầu)
        H_all[:, 0] = h_list[-1]
        for t in range(n_steps - 1):
            x_t = X[:, t, :]  # (N, F)
            h_list, c_list, h_top = self._step(x_t, h_list, c_list)
            r_t = self.readout(h_top)
            y_t = x_t + r_t

            H_all[:, t + 1] = h_top
            r_all[:, t] = r_t
            y_all[:, t] = y_t

        # Rollout autoregressive nếu cần
        if horizon > 0:
            x_t = X[:, -1, :]
            for s in range(horizon):
                h_list, c_list, h_top = self._step(x_t, h_list, c_list)
                r_t = self.readout(h_top)
                y_t = x_t + r_t

                idx = (n_steps - 1) + s
                y_all[:, idx] = y_t
                r_all[:, idx] = r_t
                H_all[:, n_steps + s] = h_top

                x_t = y_t  # autoregressive

        return y_all, r_all, H_all

    def forward_loss(
        self,
        X: torch.Tensor,
        H0: Optional[torch.Tensor] = None,
        horizon: int = 0,
        reduction: str = "mean",
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Tính loss dự báo one-step + rollout horizon (autoregressive) trên chuỗi đầu vào.

        Args:
            X: (N, T, F) chuỗi gốc (chứa full ground-truth đến T-1)
            H0: (N, d) latent init (tuỳ chọn)
            horizon: số bước rollout ngoài quan sát cuối cùng
                    (nếu >0, ta cắt input để tránh nhìn thấy tương lai)
            reduction: 'mean' | 'sum' | 'none' cho F.mse_loss

        Returns:
            loss: scalar tensor
            y_pred: (N, T-1, F) dự báo X_{t+1} cho toàn bộ t=0..T-2
        """
        device = next(self.parameters()).device
        X = X.to(device)
        N, T, Fdim = X.shape

        if horizon < 0:
            raise ValueError("horizon must be >= 0")
        if horizon >= T:
            raise ValueError(f"horizon={horizon} must be < sequence length T={T}")

        # Cắt input nếu rollout > 0 để giữ đúng số target (T-1)
        X_in = X[:, : T - horizon, :] if horizon > 0 else X

        # forward() trả (N, (T-horizon)-1 + horizon, F) = (N, T-1, F)
        y_pred, _, _ = self.forward(X_in, H0=H0, horizon=horizon)

        # Ground truth luôn là X_{1:T}
        target = X[:, 1:, :]  # (N, T-1, F)

        loss = F.mse_loss(y_pred, target, reduction=reduction)
        return loss

# Eval & Train

In [6]:
from sklearn.metrics import *

def eval(args, model, training_data_np, testing_data_np, device='cuda'):
    batch = torch.tensor(training_data_np[:, -args.seq_len:]).float().to(device)
    labels = torch.tensor(testing_data_np).float().to(device)
    horizon = labels.shape[1]
    model.eval().to(device)
    with torch.no_grad():
        y_all, r_all, H_all = model(batch, horizon=horizon)
    y_pred = y_all[:, -horizon:, 0].detach().cpu().numpy() # Adj Close
    y_gt = testing_data_np[:, :, 0] # Adj Close
    return {
        'mse': mean_squared_error(y_gt, y_pred), 
        'raw_mse': mean_squared_error(np.exp(y_gt), np.exp(y_pred)), 
        'mae': mean_absolute_error(y_gt, y_pred), 
        'raw_mae': mean_absolute_error(np.exp(y_gt), np.exp(y_pred)), 
        'r2': r2_score(y_gt, y_pred),
        'raw_r2': r2_score(np.exp(y_gt), np.exp(y_pred)), 
    }

In [7]:
myrnn = MyRNN(255, 6, args)
eval(args, myrnn, training_data_np, testing_data_np, args.device)

{'mse': 5.4592733,
 'raw_mse': 109073970.0,
 'mae': 2.0061402,
 'raw_mae': 2685.9885,
 'r2': -5.5758819580078125,
 'raw_r2': -1417.062255859375}

In [8]:
from tqdm import tqdm
import time

class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.avg == 0:
            self.avg = val
            return
        self.avg = 0.95 * self.avg + 0.05 * val

def train(args, train_loader, model: myrnn, optimizer, scheduler):
    # if args.amp:
    #     from apex import amp
    global best_loss
    test_losses = []
    end = time.time()

    step = 0
    for epoch in range(args.epochs):
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        stats01 = AverageMeter()
        stats02 = AverageMeter()
        p_bar = tqdm(train_loader)
        for batch_idx, samples in enumerate(p_bar):
            step += 1
            model.train().to(args.device)
          
            samples = samples[0].float().to(args.device)
            data_time.update(time.time() - end)

            loss = model.forward_loss(samples, horizon=args.horizon)
            
            loss.backward()
            optimizer.step()
            scheduler.step()

            losses.update(loss.item())

            batch_time.update(time.time() - end)
            end = time.time()
            # mask_probs.update(mask.mean().item())
            p_bar.set_description(
                "Ep: {epoch}/{epochs:3}. LR: {lr:.3e}. "
                "Loss: {loss:.4f}.".format(
                epoch=epoch + 1,
                epochs=args.epochs,
                lr=scheduler.get_last_lr()[0],
                data=data_time.avg,
                bt=batch_time.avg,
                loss=losses.avg,
            ))
            p_bar.update()
            
            if (step + 1) % args.eval_steps == 0:
                test_model = model

                test_metrics = eval(args, model, training_data_np, testing_data_np, args.device)
                print(test_metrics)
                test_loss = test_metrics['mse']

                is_best = test_loss < best_loss
                best_loss = min(test_loss, best_loss)


                test_losses.append(test_loss)
                print('Best loss: {:.3f}'.format(best_loss))
                print('Mean loss: {:.3f}\n'.format(
                    np.mean(test_losses[-20:])))


In [9]:
import math
import copy
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR

def get_cosine_schedule_with_warmup(optimizer,
                                    num_warmup_steps,
                                    num_training_steps,
                                    num_cycles=7./16.,
                                    last_epoch=-1):
    def _lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        no_progress = float(current_step - num_warmup_steps) / \
            float(max(1, num_training_steps - num_warmup_steps))
        return max(0., math.cos(math.pi * num_cycles * no_progress))

    return LambdaLR(optimizer, _lr_lambda, last_epoch)

In [10]:
training_data_np, testing_data_np = get_data_splits()
train_loader = DataLoader(StockDataset(training_data_np, args.seq_len), batch_size=1, shuffle=True)

myrnn = MyRNN(args.n_nodes, args.n_feats, args)

from torch.optim import Adam

optimizer = Adam(myrnn.parameters(), lr=args.lr)
total_steps = args.epochs * len(train_loader)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=args.warmup,
    num_training_steps=total_steps,
)

In [11]:
print(f'{get_model_size(myrnn) * 1e3:.2f}K')

14.53K


In [12]:
best_loss = 9999

train(args, train_loader, myrnn, optimizer, scheduler)

Ep: 1/  5. LR: 9.998e-05. Loss: 0.0887.:   7%|▋         | 290/3999 [00:16<03:08, 19.71it/s]

{'mse': 0.012425347, 'raw_mse': 3213.3289, 'mae': 0.081988014, 'raw_mae': 18.606659, 'r2': 0.9850150942802429, 'raw_r2': 0.9576401114463806}
Best loss: 0.012
Mean loss: 0.012



Ep: 1/  5. LR: 9.994e-05. Loss: 0.0490.:  14%|█▍        | 557/3999 [00:34<04:43, 12.15it/s]

{'mse': 0.00756226, 'raw_mse': 1689.2867, 'mae': 0.06391531, 'raw_mae': 14.509393, 'r2': 0.9908967614173889, 'raw_r2': 0.9778246283531189}
Best loss: 0.008
Mean loss: 0.010



Ep: 1/  5. LR: 9.986e-05. Loss: 0.0609.:  21%|██        | 825/3999 [00:51<03:41, 14.35it/s]

...

Ep: 5/  5. LR: 2.141e-05. Loss: 0.0392.:  96%|█████████▌| 3838/3999 [03:56<00:10, 15.44it/s]

{'mse': 0.013779279, 'raw_mse': 812.1551, 'mae': 0.092964314, 'raw_mae': 15.121838, 'r2': 0.9834080934524536, 'raw_r2': 0.9893359541893005}
Best loss: 0.003
Mean loss: 0.069



Ep: 5/  5. LR: 1.968e-05. Loss: 0.0380.: : 4100it [04:12, 14.31it/s]                        

{'mse': 0.00332369, 'raw_mse': 511.16443, 'mae': 0.04234987, 'raw_mae': 8.560571, 'r2': 0.996005117893219, 'raw_r2': 0.993163526058197}
Best loss: 0.003
Mean loss: 0.069



Ep: 5/  5. LR: 1.951e-05. Loss: 0.0387.: 100%|██████████| 3999/3999 [04:14<00:00, 15.73it/s]
